![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/master/assets/img/ODPi_Egeria_Logo_color.png)

### Egeria Hands-On Lab
# Welcome to the Understanding Cohort Configuration Lab

## Introduction

Egeria is an open source project that provides open standards and implementation libraries to connect tools,
catalogs and platforms together so they can share information (called metadata) about data and the technology that supports it.

The ODPi Egeria repository services provide APIs for understanding the make up of the cohorts that an OMAG Server
is connected to. 
This hands-on lab steps through each of the repository services operations for understanding a cohort, providing a explaination and the code to call each operation.

## The Scenario

Gary Geeke is the IT Infrastructure leader at Coco Pharmaceuticals.  He has set up a number of OMAG Servers and
is validating they are operating correctly.  Gary's userId is `garygeeke`.

![Gary Geeke](https://raw.githubusercontent.com/odpi/data-governance/master/docs/coco-pharmaceuticals/personas/gary-geeke.png)

In the **[Server Configuration Lab](../egeria-server-config.ipynb)**, Gary configured servers for the OMAG Server Platforms shown in Figure 1:

![Figure 1](../images/coco-pharmaceuticals-systems-omag-server-platforms.png)
> **Figure 1:** Coco Pharmaceuticals' OMAG Server Platforms

The following command ensures these platforms, and the metadata servers that run on them, are started.  Click on the play triangle at the top of the page to run it.

In [ ]:
%run ../common/environment-check.ipynb

----
Figure 2 shows which metadata servers belong to each cohort.

![Figure 2](../images/coco-pharmaceuticals-systems-cohorts.png)
> **Figure 2:** Membership of Coco Pharmaceuticals' cohorts

The open metadata repository cohort protocols are peer-to-peer.  This means that each member of
the cohort maintains its own view of the other members of the cohort.  This information is
stored in the **cohort registry store**.  All of the queries that follow are being made to the
cohort registry stores of Coco Pharmaceuticals metadata servers.

## Querying a server's cohorts

The code below queries each server's cohort registry store and retrieves the names of the cohort that it is connected to.


In [ ]:

print ("  ")
print ('Cohort(s) for cocoMDS1 are [%s]' % ', '.join(map(str, queryServerCohorts(cocoMDS1Name, cocoMDS1PlatformName, cocoMDS1PlatformURL))))
print ('Cohort(s) for cocoMDS2 are [%s]' % ', '.join(map(str, queryServerCohorts(cocoMDS2Name, cocoMDS2PlatformName, cocoMDS2PlatformURL))))
print ('Cohort(s) for cocoMDS3 are [%s]' % ', '.join(map(str, queryServerCohorts(cocoMDS3Name, cocoMDS3PlatformName, cocoMDS3PlatformURL))))
print ('Cohort(s) for cocoMDS4 are [%s]' % ', '.join(map(str, queryServerCohorts(cocoMDS4Name, cocoMDS4PlatformName, cocoMDS4PlatformURL))))
print ('Cohort(s) for cocoMDS5 are [%s]' % ', '.join(map(str, queryServerCohorts(cocoMDS5Name, cocoMDS5PlatformName, cocoMDS5PlatformURL))))
print ('Cohort(s) for cocoMDS6 are [%s]' % ', '.join(map(str, queryServerCohorts(cocoMDS6Name, cocoMDS6PlatformName, cocoMDS6PlatformURL))))
print ('Cohort(s) for cocoMDSx are [%s]' % ', '.join(map(str, queryServerCohorts(cocoMDSxName, cocoMDSxPlatformName, cocoMDSxPlatformURL))))
print ("  ")


----
A quick check shows that the results of the query matches the diagram in figure 2.


## Querying local registration

The local registration describes the registration information that the metadata server
has shared with the cohorts it has connected to. The command below retrieves the
local registration information.  Here we are looking at cocoMDS2.

In [ ]:

printLocalRegistration(cocoMDS2Name, cocoMDS2PlatformName, cocoMDS2PlatformURL)


----
If we add in the name of the cohort, it is possible to see the time that it first registered
with that cohort.

In [ ]:

printLocalRegistrationForCohort(cocoMDS2Name, cocoCohort, cocoMDS2PlatformName, cocoMDS2PlatformURL)
print(" ")
printLocalRegistrationForCohort(cocoMDS2Name, devCohort, cocoMDS2PlatformName, cocoMDS2PlatformURL)
print(" ")
printLocalRegistrationForCohort(cocoMDS2Name, iotCohort, cocoMDS2PlatformName, cocoMDS2PlatformURL)


----
The times of registration are pretty close in this example because all of the cohorts were in the initial configuration for this server.  If the registration time shows as blank it means that the server has not
registered with the cohort.


## Querying remote members

Finally each cohort registry store lists all of the remote members of the cohort that a server has exchanged
registration information with.  These are the remote members from cocoMDS2's perspective.

In [ ]:

print("Cohort " + cocoCohort + "...")
printRemoteRegistrations(cocoMDS2Name, cocoCohort, cocoMDS2PlatformName, cocoMDS2PlatformURL)
print(" ")

print("Cohort " + devCohort + "...")
printRemoteRegistrations(cocoMDS2Name, devCohort, cocoMDS2PlatformName, cocoMDS2PlatformURL)
print(" ")

print("Cohort " + iotCohort + "...")
printRemoteRegistrations(cocoMDS2Name, iotCohort, cocoMDS2PlatformName, cocoMDS2PlatformURL)
print(" ")


----